In [2]:
import json
import pandas as pd
import altair as alt
import geopandas as gpd
import re
from data_cleaner import *
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [97]:
raw = pd.read_csv("acled.csv")

In [98]:
raw['Region'] = raw['Region'].replace('South Asia', 'Asia')
raw['Region'] = raw['Region'].replace('Southeast Asia', 'Asia')
raw['Region'] = raw['Region'].replace('Caucasus and Central Asia', 'Asia')
raw['Region'] = raw['Region'].replace('Southern Africa', 'Africa')
raw['Region'] = raw['Region'].replace('Western Africa', 'Africa')
raw['Region'] = raw['Region'].replace('Eastern Africa', 'Africa')
raw['Region'] = raw['Region'].replace('Northern Africa', 'Africa')
raw['Region'] = raw['Region'].replace('Middle Africa', 'Africa')
raw['Region'] = raw['Region'].replace('Central America', 'Americas')
raw['Region'] = raw['Region'].replace('Caribbean', 'Americas')
raw['Region'] = raw['Region'].replace('North America', 'Americas')
raw['Region'] = raw['Region'].replace('South America', 'Americas')
raw['Country'] = raw['Country'].replace('Democratic Republic of Congo', 'D.R. Congo')

In [99]:
regions = raw["Region"].value_counts()
regions

Region
Africa         49
Middle East    35
Asia           25
Americas       23
Europe         12
Name: count, dtype: int64

In [101]:
regions = raw["Region"].value_counts()
countries = raw["Country"].value_counts()
event_types = raw["event_type"].value_counts()
#sub_event_types = raw["sub_event_type"].value_counts()

In [103]:
violence_object = {}

for region, value in regions.items():
    country_subset = raw[raw["Region"] == region]
    violence_object[region] = {}
    violence_object[region]["name"] = region
    violence_object[region]["violent_events"] = value
    region_countries = country_subset["Country"].value_counts()

    for country, value in region_countries.items():
        violence_object[region][country] = {}
        violence_object[region][country]["name"] = country
        violence_object[region][country]["violent_events"] = value

In [ ]:
violence_object

In [ ]:
with open("samplebubble.json", "w") as outfile: 
    json.dump(violence_object, outfile)

In [8]:
#different data shape:
raw2 = pd.read_csv("datatest.csv")

In [105]:
testing = raw.value_counts()

In [ ]:
violence_array = []

for id, value in testing.items():
    row = {}
    row["Country"] = id[0]
    row['Region'] = id[1]
    row['event_type'] = id[2]
    row['sub_event_type'] = id[3]
    row['violent_events'] = value
    violence_array.append(row)

violence_array

In [21]:
with open("arraybubble.json", "w") as outfile: 
    json.dump(violence_array, outfile)

In [127]:
raw = pd.read_csv("../data/acled_2023.csv")

In [129]:
raw['region'] = raw['region'].replace('South Asia', 'Asia')
raw['region'] = raw['region'].replace('Southeast Asia', 'Asia')
raw['region'] = raw['region'].replace('Caucasus and Central Asia', 'Asia')
raw['region'] = raw['region'].replace('Southern Africa', 'Africa')
raw['region'] = raw['region'].replace('Western Africa', 'Africa')
raw['region'] = raw['region'].replace('Eastern Africa', 'Africa')
raw['region'] = raw['region'].replace('Northern Africa', 'Africa')
raw['region'] = raw['region'].replace('Middle Africa', 'Africa')
raw['region'] = raw['region'].replace('Central America', 'Americas')
raw['region'] = raw['region'].replace('Caribbean', 'Americas')
raw['region'] = raw['region'].replace('North America', 'Americas')
raw['region'] = raw['region'].replace('South America', 'Americas')
raw['country'] = raw['country'].replace('Democratic Republic of Congo', 'D.R. Congo')

In [130]:
acled = raw
acled = acled[acled["event_type"] != "Protests"]
acled = acled[acled["sub_event_type"] != "Headquarters or base established"]
acled = acled[acled["sub_event_type"] != "Other"]
acled = acled[acled["sub_event_type"] != "Change to group/activity"]
acled = acled[acled["sub_event_type"] != "Agreement"]
acled = acled[acled["sub_event_type"] != "Non-violent transfer of territory"]
acled = acled[acled['year'] == 2023]
good_cols = acled[['region','country','event_type']]

In [132]:
big, small, all = small_big_split(good_cols)

top30 = top_k_countries(all, 30)

In [ ]:
counter = top30.value_counts()

acled_array = []

for id, value in counter.items():
    row = {}
    row["Country"] = id[1]
    row['Region'] = id[0]
    row['event_type'] = id[2]
    row['conflict_size'] = id[3]
    row['violent_events'] = value
    acled_array.append(row)

acled_array

In [118]:
mig = pd.read_csv("../data/migration_wb.csv")
mig["Country_Name"] = mig["Country Name"].astype(str)

In [ ]:
migra = mig[mig["Series Name"] == "Net migration"]
pop = mig[mig["Series Name"] == "Population, total"]

wb_data = migra.merge(pop, how='left', on='Country_Name')
wb_data.head()

wb_data["Country"] = wb_data["Country Name_x"]
wb_data["net_migration"] = wb_data["2023 [YR2023]_x"]
wb_data["population"] = wb_data["2023 [YR2023]_y"]

wb_final = wb_data[['Country','population','net_migration']]


wb_final['mg_per_1000'] = 1000 * wb_final['net_migration'] / wb_final['population']
wb_final.head()

In [120]:
counter_wb = wb_final.value_counts()
wb_array = []

for id, row in counter_wb.items():
    row = {}
    row["Country"] = id[0]
    row["pop"] = id[1]
    row["mig"] = id[2]
    row["mig_1000"] = id[3]
    wb_array.append(row)


In [134]:
#bubble chart file
with open("acledArray.json", "w") as outfile: 
    json.dump(acled_array, outfile)

In [135]:
#Migration Chart file
wb_final.to_csv('wbFinal.csv', index=False)

#Bar Chart File
top30df = pd.DataFrame(acled_array)
top30df.to_csv('acled.csv', index=False)

In [ ]:
# Getting the average migration per 1000 ppl
test = top30df.merge(wb_final, how='left')
eye = test[test['mg_per_1000'] > 10]
eye

yo = test[test['Country'] == 'Syria']
yo

In [84]:
yup = test['mg_per_1000'].value_counts()
yup.index

Index([   32.59579556803987, -0.39110021599920225,   -0.967908977839724,
       -0.20784050287798506,  0.02772355691793632,  -0.6412474243054395,
        -0.1318549173972894,   -33.12165082472128, -0.34028180511299533,
       -0.26807309332185764,  -1.6534958374344624,  -1.5588595547702413,
         -3.360860811661393, -0.16755509848696698,  -1.7922478131349817,
        -3.7276680027189837, -0.09483346882477156, -0.18148627312239474,
        -1.7171167927050484,  -1.0751141271191926,    1.024834216487132,
        -0.6902199548215886,    48.23562162162162,   0.9682444585525377,
        -0.5175421508540496,  -2.7131465258615464,   0.9939969688820757],
      dtype='float64', name='mg_per_1000')

In [85]:
x = [   32.59579556803987, -0.39110021599920225,   -0.967908977839724,
       -0.20784050287798506,  0.02772355691793632,  -0.6412474243054395,
        -0.1318549173972894,   -33.12165082472128, -0.34028180511299533,
       -0.26807309332185764,  -1.6534958374344624,  -1.5588595547702413,
         -3.360860811661393, -0.16755509848696698,  -1.7922478131349817,
        -3.7276680027189837, -0.09483346882477156, -0.18148627312239474,
        -1.7171167927050484,  -1.0751141271191926,    1.024834216487132,
        -0.6902199548215886,    48.23562162162162,   0.9682444585525377,
        -0.5175421508540496,  -2.7131465258615464,   0.9939969688820757]

x2 = [   0, -0.39110021599920225,   -0.967908977839724,
       -0.20784050287798506,  0.02772355691793632,  -0.6412474243054395,
        -0.1318549173972894,   0, -0.34028180511299533,
       -0.26807309332185764,  -1.6534958374344624,  -1.5588595547702413,
         -3.360860811661393, -0.16755509848696698,  -1.7922478131349817,
        -3.7276680027189837, -0.09483346882477156, -0.18148627312239474,
        -1.7171167927050484,  -1.0751141271191926,    1.024834216487132,
        -0.6902199548215886,    0,   0.9682444585525377,
        -0.5175421508540496,  -2.7131465258615464,   0.9939969688820757]
y = sum(x)/len(x)
y2 = sum(x2)/len(x2)
print(y, y2)

1.0565226747188805 -0.7105057091677938
